In [ ]:
import json
import numpy as np

In [ ]:
with open('../data/gold_set.txt', "r") as f:
    gold_set = f.read().split('\n')[:-1]
f.close()

In [ ]:
query_len = 3
query_per_set = 10
query_list = []
for st in gold_set:
    json_st = json.loads(st)
    count = 0
    for i in range(query_per_set):
        q = {}
        q['title'] = json_st['title'].lower()
        q['qid'] = '{}|{}|{}'.format(json_st['id'], query_len, count)
        q['entities'] = np.random.choice(json_st['entities'], query_len, replace=False).tolist()
        query_list.append(json.dumps(q))
        count += 1

In [ ]:
with open('../data/query-3.txt', "w+") as f:
    f.write('\n'.join(query_list))
f.close()

In [ ]:
from textacy.similarity import jaccard

In [ ]:
jaccard("Boston is a city of Maryland.","Amherst localed in Maryland")

In [ ]:
import pandas as pd
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.corpus import wordnet as wn
import itertools

#use stemmer
stm = PorterStemmer()
sent1 =  "_ is a city of Maryland."
sent2 = "Maryland has a city called _"
#Convert the tag given by nltk.pos_tag to the tag used by wordnet.synsets
tag_dict = {'N': 'n', 'J': 'a', 'R': 'r', 'V': 'v'}

s1 = nltk.pos_tag(nltk.word_tokenize(sent1))

s1 = dict(filter(lambda x: len(x[1])>0,
                 map(lambda row: (row[0],wn.synsets(
                       stm.stem(row[0]),
                       tag_dict[row[1][0]])) if row[1][0] in tag_dict.keys() 
                     else (row[0],[]),s1)))

s2 = nltk.pos_tag(nltk.word_tokenize(sent2))

s2 = dict(filter(lambda x: len(x[1])>0,
                 map(lambda row: (row[0],wn.synsets(
                          stm.stem(row[0]),
                          tag_dict[row[1][0]])) if row[1][0] in tag_dict.keys() 
                     else (row[0],[]),s2)))

In [ ]:
res = {}
for w2,gr2 in s2.items():
    for w1,gr1 in s1.items():
        tmp = pd.Series(list(map(lambda row: row[1].path_similarity(row[0]),
                                 itertools.product(gr1,gr2)))).dropna()
        if len(tmp)>0:
            res[(w1,w2)] = tmp.max()
print(res)

In [ ]:
similarity = pd.Series(res).groupby(level=0).max().mean()
print(similarity)

In [ ]:
from bert_serving.client import BertClient
bc = BertClient(ip='localhost')
bc.encode(['First do it', 'then do it right', 'then do it better'])